In [2]:
import json
import pandas as pd
from datetime import datetime
from listas import charlson_dict, delirium_list, dementia_list, depresyndr_list, uriincont_list, fecincont_list, pressulc_list, immob_list, conf_list, osteopor_list, sarcopenia_list, sleepdisturb_list, chrpain_list, iatrog_list, constipation_list, CVdisease_list, heartdisease_list, ND_list, DM_list, hepatopat_list, neopl_list, AcuteRenalF_list, dizsyn_list, VIH_list, psicosis_list

from auxiliary_functions import (obtenir_data_presencia_codi, restar_dates, codis_ICD, nombre_ingressos,
                                 dies_ingressat_total, interval_10_edat,
                                 sumar_barthel, sumar_emina, obtenir_ultima_clau, obtenir_pes_o_mitjana,
                                 canadenca_comparada, disfagia_mecvvs, extreure_valors_claus,
                                 extreure_valors_claus_simple, obtenir_valors_clau_interes, index_charlson,
                                 obtenir_pes_mes_antic, obtenir_pes_mes_nou, obtenir_data_pes_mes_antic,
                                 obtenir_primera_data_mecvv, obtenir_pes_coincident_mecvv, restar_columnes_object)

taula = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


In [3]:
taula.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,Pes més antic,Pes més nou,Data pes més antic,Data primer MECVV,Pes coincident primer MECVV,Pèrdua pes entre ingressos,Pèrdua pes total,Data més antiga pneumònia,Dies entre primer ICD pneumònia i primer MECVV positiu
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,NaT,None,None,None,2020-06-18,NaN
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,>90,70,53.7,2022-07-06,NaT,None,None,16.3,2022-09-29,NaN
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,NaT,None,None,None,2020-07-08,NaN
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,49.900,49.900,2022-06-02,NaT,None,None,0.0,2022-08-07,NaN


In [4]:
AMB_PA = taula[taula["PA diagnosticada"] == 1.0]
AMB_PA_MECVV = taula[
    (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] < 30)]
SENSE_PA = taula[
           (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] > 30) &
           (taula['P diagnosticada'] == 1.0)]
TOTAL = taula

# Síndromes geriàtrics

## Index de Charlson

In [5]:
Mitjana_charlson = AMB_PA['Charlson'].mean()
desviacio_estandard_charlson = AMB_PA['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 10.24 ± 10.73


In [6]:
Mitjana_charlson = AMB_PA_MECVV['Charlson'].mean()
desviacio_estandard_charlson = AMB_PA_MECVV['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 16.00 ± 11.60


In [262]:
Mitjana_charlson = SENSE_PA['Charlson'].mean()
desviacio_estandard_charlson = SENSE_PA['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 16.50 ± 12.83


In [261]:
Mitjana_charlson = TOTAL['Charlson'].mean()
desviacio_estandard_charlson = TOTAL['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 11.62 ± 13.16


#### Pvalor

##### Normalitat

In [8]:
stat, p_value = shapiro(AMB_PA['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.7994
Valor p = 0.0020
El nombre de files no segueix una distribució normal.


In [9]:
stat, p_value = shapiro(AMB_PA_MECVV['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9651
Valor p = 0.8571
El nombre de files segueix una distribució normal


In [10]:
stat, p_value = shapiro(SENSE_PA['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9177
Valor p = 0.4113
El nombre de files segueix una distribució normal


In [11]:
stat, p_value = shapiro(TOTAL['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.7836
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [12]:
stat, p_value_12 = mannwhitneyu(AMB_PA['Charlson'], AMB_PA_MECVV['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.23085687112868614


In [263]:
stat, p_value_13 = mannwhitneyu(AMB_PA['Charlson'], SENSE_PA['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.1977214805403812


In [264]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['Charlson'], SENSE_PA['Charlson'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.9360013644802581


In [266]:
stat, p_value_14 = mannwhitneyu(AMB_PA['Charlson'], TOTAL['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL", p_value_14)

P-valor entre AMB_PA i TOTAL 0.8623665948573777


In [267]:
stat, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Charlson'], TOTAL['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL", p_value_24)

P-valor entre AMB_PA_MECVV i TOTAL 0.2016616772763471


In [268]:
stat, p_value_34 = mannwhitneyu(SENSE_PA['Charlson'], TOTAL['Charlson'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL", p_value_34)

P-valor entre SENSE_PA i TOTAL 0.19370672249585386


## Caigudes prèvies

In [269]:
caprev_AMB_PA= (AMB_PA['Caigudes prèvies'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_AMB_PA:.2f}%")

Percentatge de caigudes prèvies: 5.88%


In [271]:
caprev_AMB_PA_MECVV= (AMB_PA_MECVV['Caigudes prèvies'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_AMB_PA_MECVV:.2f}%")

Percentatge de caigudes prèvies: 0.00%


In [273]:
caprev_SENSE_PA= (SENSE_PA['Caigudes prèvies'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_SENSE_PA:.2f}%")

Percentatge de caigudes prèvies: 12.50%


In [274]:
caprev_TOTAL= (TOTAL['Caigudes prèvies'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_TOTAL:.2f}%")

Percentatge de caigudes prèvies: 6.00%


#### Pvalor

##### Normalitat

In [275]:
stat, p_value = shapiro(AMB_PA['Caigudes prèvies'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [276]:
stat, p_value = shapiro(AMB_PA_MECVV['Caigudes prèvies'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [278]:
stat, p_value = shapiro(SENSE_PA['Caigudes prèvies'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4184
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [279]:
stat, p_value = shapiro(TOTAL['Caigudes prèvies'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2548
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [280]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Caigudes prèvies'], columns=AMB_PA_MECVV['Caigudes prèvies'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.


In [281]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Caigudes prèvies'], columns=SENSE_PA['Caigudes prèvies'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.


In [282]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Caigudes prèvies'], columns=SENSE_PA['Caigudes prèvies'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [284]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Caigudes prèvies'], columns=TOTAL['Caigudes prèvies'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 3.7353515625
P-valor entre AMB_PA i TOTAL: 0.053272454982386926
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i TOTAL.


In [285]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Caigudes prèvies'], columns=TOTAL['Caigudes prèvies'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i TOTAL.


In [286]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Caigudes prèvies'], columns=TOTAL['Caigudes prèvies'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 1.469387755102041
P-valor entre SENSE_PA i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de sexe en SENSE_PA i TOTAL.


## Deliris

In [287]:
del_AMB_PA= (AMB_PA['Deliris'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_AMB_PA:.2f}%")

Percentatge de deliris: 23.53%


In [288]:
del_AMB_PA_MECVV= (AMB_PA_MECVV['Deliris'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_AMB_PA_MECVV:.2f}%")

Percentatge de deliris: 25.00%


In [289]:
del_SENSE_PA= (SENSE_PA['Deliris'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_SENSE_PA:.2f}%")

Percentatge de deliris: 62.50%


In [290]:
del_TOTAL= (TOTAL['Deliris'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_TOTAL:.2f}%")

Percentatge de deliris: 20.00%


#### Pvalor

##### Normalitat

In [291]:
stat, p_value = shapiro(AMB_PA['Deliris'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [292]:
stat, p_value = shapiro(AMB_PA_MECVV['Deliris'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [293]:
stat, p_value = shapiro(SENSE_PA['Deliris'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6412
Valor p = 0.0005
El nombre de files no segueix una distribució normal.


In [294]:
stat, p_value = shapiro(TOTAL['Deliris'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4904
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [295]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Deliris'], columns=AMB_PA_MECVV['Deliris'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i AMB_PA_MECVV.


In [296]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Deliris'], columns=SENSE_PA['Deliris'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA i SENSE_PA.


In [299]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Deliris'], columns=SENSE_PA['Deliris'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [300]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Deliris'], columns=TOTAL['Deliris'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 11.89654215976331
P-valor entre AMB_PA i TOTAL: 0.0005623824703676534
Hi ha diferències significatives entre les distribucions de sexe en AMB_PA i TOTAL.


In [301]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Deliris'], columns=TOTAL['Deliris'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de sexe en AMB_PA_MECVV i TOTAL.


In [302]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Deliris'], columns=TOTAL['Deliris'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.302222222222222
P-valor entre SENSE_PA i TOTAL: 0.0380626070324449
Hi ha diferències significatives entre les distribucions de sexe en SENSE_PA i TOTAL.


## Demència

In [303]:
dem_AMB_PA= (AMB_PA['Demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_AMB_PA:.2f}%")

Percentatge de demència: 47.06%


In [304]:
dem_AMB_PA_MECVV= (AMB_PA_MECVV['Demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_AMB_PA_MECVV:.2f}%")

Percentatge de demència: 25.00%


In [305]:
dem_SENSE_PA= (SENSE_PA['Demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_SENSE_PA:.2f}%")

Percentatge de demència: 62.50%


In [306]:
dem_TOTAL= (TOTAL['Demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_TOTAL:.2f}%")

Percentatge de demència: 32.00%


#### Pvalor

##### Normalitat

In [307]:
stat, p_value = shapiro(AMB_PA['Demència'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6418
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [308]:
stat, p_value = shapiro(AMB_PA_MECVV['Demència'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [330]:
stat, p_value = shapiro(SENSE_PA['Demència'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6412
Valor p = 0.0005
El nombre de files no segueix una distribució normal.


In [310]:
stat, p_value = shapiro(TOTAL['Demència'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5879
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [311]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Demència'], columns=AMB_PA_MECVV['Demència'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de demència en AMB_PA i AMB_PA_MECVV.


In [313]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Demència'], columns=SENSE_PA['Demència'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de demència en AMB_PA i SENSE_PA.


In [314]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Demència'], columns=SENSE_PA['Demència'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [315]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Demència'], columns=TOTAL['Demència'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 13.223042052469136
P-valor entre AMB_PA i TOTAL: 0.00027652843917458906
Hi ha diferències significatives entre les distribucions de demència en AMB_PA i TOTAL.


In [316]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Demència'], columns=TOTAL['Demència'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i TOTAL.


In [326]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Demència'], columns=TOTAL['Demència'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.302222222222222
P-valor entre SENSE_PA i TOTAL: 0.0380626070324449
Hi ha diferències significatives entre les distribucions de demència en SENSE_PA i TOTAL.


## Síndrome depressiu

In [318]:
depre_AMB_PA= (AMB_PA['Síndrome depressiu'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_AMB_PA:.2f}%")

Percentatge de síndrome depressiu: 23.53%


In [319]:
depre_AMB_PA_MECVV= (AMB_PA_MECVV['Síndrome depressiu'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_AMB_PA_MECVV:.2f}%")

Percentatge de síndrome depressiu: 12.50%


In [320]:
depre_SENSE_PA= (SENSE_PA['Síndrome depressiu'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_SENSE_PA:.2f}%")

Percentatge de síndrome depressiu: 12.50%


In [321]:
depre_TOTAL= (TOTAL['Síndrome depressiu'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_TOTAL:.2f}%")

Percentatge de síndrome depressiu: 14.00%


#### Pvalor

##### Normalitat

In [322]:
stat, p_value = shapiro(AMB_PA['Síndrome depressiu'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [323]:
stat, p_value = shapiro(AMB_PA_MECVV['Síndrome depressiu'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4184
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [324]:
stat, p_value = shapiro(SENSE_PA['Síndrome depressiu'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4184
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [325]:
stat, p_value = shapiro(TOTAL['Síndrome depressiu'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4124
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [327]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Síndrome depressiu'], columns=AMB_PA_MECVV['Síndrome depressiu'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.9600000000000002
P-valor entre AMB_PA i AMB_PA_MECVV: 0.32718687779030275
No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i AMB_PA_MECVV.


In [328]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Síndrome depressiu'], columns=SENSE_PA['Síndrome depressiu'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i SENSE_PA.


In [329]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Síndrome depressiu'], columns=SENSE_PA['Síndrome depressiu'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [332]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Síndrome depressiu'], columns=TOTAL['Síndrome depressiu'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 11.89654215976331
P-valor entre AMB_PA i TOTAL: 0.0005623824703676534
Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i TOTAL.


In [333]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Síndrome depressiu'], columns=TOTAL['Síndrome depressiu'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 1.469387755102041
P-valor entre AMB_PA_MECVV i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i TOTAL.


In [334]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Síndrome depressiu'], columns=TOTAL['Síndrome depressiu'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 1.469387755102041
P-valor entre SENSE_PA i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de síndrome depressiu en SENSE_PA i TOTAL.


## Incontinència urinària

In [335]:
ic_AMB_PA= (AMB_PA['Incont.uri'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència urinària: {ic_AMB_PA:.2f}%")

Percentatge d'incontinència urinària: 29.41%


In [336]:
ic_AMB_PA_MECVV= (AMB_PA_MECVV['Incont.uri'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència urinària: {ic_AMB_PA_MECVV:.2f}%")

Percentatge d'incontinència urinària: 50.00%


In [337]:
ic_SENSE_PA= (SENSE_PA['Incont.uri'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència urinària: {ic_SENSE_PA:.2f}%")

Percentatge d'incontinència urinària: 0.00%


In [338]:
ic_TOTAL= (TOTAL['Incont.uri'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència urinària: {ic_TOTAL:.2f}%")

Percentatge d'incontinència urinària: 16.00%


#### Pvalor

##### Normalitat

In [339]:
stat, p_value = shapiro(AMB_PA['Incont.uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5785
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [340]:
stat, p_value = shapiro(AMB_PA_MECVV['Incont.uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6647
Valor p = 0.0009
El nombre de files no segueix una distribució normal.


In [341]:
stat, p_value = shapiro(SENSE_PA['Incont.uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [344]:
stat, p_value = shapiro(TOTAL['Incont.uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4412
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [345]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Incont.uri'], columns=AMB_PA_MECVV['Incont.uri'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.6666666666666665
P-valor entre AMB_PA i AMB_PA_MECVV: 0.10247043485974942
No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.


In [346]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Incont.uri'], columns=SENSE_PA['Incont.uri'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.


In [347]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Incont.uri'], columns=SENSE_PA['Incont.uri'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [348]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Incont.uri'], columns=TOTAL['Incont.uri'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 12.524513888888887
P-valor entre AMB_PA i TOTAL: 0.00040164738150225485
Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i TOTAL.


In [349]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Incont.uri'], columns=TOTAL['Incont.uri'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 4.5
P-valor entre AMB_PA_MECVV i TOTAL: 0.033894853524689295
Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i TOTAL.


In [350]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Incont.uri'], columns=TOTAL['Incont.uri'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'incontinència urinària en SENSE_PA i TOTAL.


## Incontinència fecal 

In [351]:
ifec_AMB_PA= (AMB_PA['Incont.fec'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència fecal: {ifec_AMB_PA:.2f}%")

Percentatge d'incontinència fecal: 29.41%


In [352]:
ifec_AMB_PA_MECVV= (AMB_PA_MECVV['Incont.fec'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència fecal: {ifec_AMB_PA_MECVV:.2f}%")

Percentatge d'incontinència fecal: 25.00%


In [353]:
ifec_SENSE_PA= (SENSE_PA['Incont.fec'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència fecal: {ifec_SENSE_PA:.2f}%")

Percentatge d'incontinència fecal: 0.00%


In [354]:
ifec_TOTAL= (TOTAL['Incont.fec'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència fecal: {ifec_TOTAL:.2f}%")

Percentatge d'incontinència fecal: 12.00%


#### Pvalor

##### Normalitat

In [355]:
stat, p_value = shapiro(AMB_PA['Incont.fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5785
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [356]:
stat, p_value = shapiro(AMB_PA_MECVV['Incont.fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [357]:
stat, p_value = shapiro(SENSE_PA['Incont.fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [358]:
stat, p_value = shapiro(TOTAL['Incont.fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3802
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [359]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Incont.fec'], columns=AMB_PA_MECVV['Incont.fec'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i AMB_PA_MECVV.


In [360]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Incont.fec'], columns=SENSE_PA['Incont.fec'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i SENSE_PA.


In [361]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Incont.fec'], columns=SENSE_PA['Incont.fec'])

chi2_23, p_value_23, dof_23, expected_13 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [363]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Incont.fec'], columns=TOTAL['Incont.fec'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 12.524513888888887
P-valor entre AMB_PA i TOTAL: 0.00040164738150225485
Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i TOTAL.


In [364]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Incont.fec'], columns=TOTAL['Incont.fec'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i TOTAL.


In [366]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Incont.fec'], columns=TOTAL['Incont.fec'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'incontinència fecal en SENSE_PA i TOTAL.


## Úlceres de pressió

In [368]:
up_AMB_PA= (AMB_PA['Úlceres pressió'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_AMB_PA:.2f}%")

Percentatge de úlceres de pressió: 5.88%


In [369]:
up_AMB_PA_MECVV= (AMB_PA_MECVV['Úlceres pressió'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_AMB_PA_MECVV:.2f}%")

Percentatge de úlceres de pressió: 0.00%


In [370]:
up_SENSE_PA= (SENSE_PA['Úlceres pressió'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_SENSE_PA:.2f}%")

Percentatge de úlceres de pressió: 0.00%


In [371]:
up_TOTAL= (TOTAL['Úlceres pressió'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_TOTAL:.2f}%")

Percentatge de úlceres de pressió: 2.00%


#### Pvalor

##### Normalitat

In [372]:
stat, p_value = shapiro(AMB_PA['Úlceres pressió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [373]:
stat, p_value = shapiro(AMB_PA_MECVV['Úlceres pressió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [374]:
stat, p_value = shapiro(SENSE_PA['Úlceres pressió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [375]:
stat, p_value = shapiro(TOTAL['Úlceres pressió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.1254
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [382]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Úlceres pressió'], columns=AMB_PA_MECVV['Úlceres pressió'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i AMB_PA_MECVV.")


Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i AMB_PA_MECVV.


In [377]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Úlceres pressió'], columns=SENSE_PA['Úlceres pressió'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i SENSE_PA.")


Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'úlceres pressió en AMB_PA i SENSE_PA.


In [383]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Úlceres pressió'], columns=SENSE_PA['Úlceres pressió'])

chi2_23, p_value_23, dof_23, expected_13 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [379]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Úlceres pressió'], columns=TOTAL['Úlceres pressió'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i TOTAL.")


Estadístic xi-quadrat entre AMB_PA i TOTAL: 3.7353515625
P-valor entre AMB_PA i TOTAL: 0.053272454982386926
No hi ha diferències significatives entre les distribucions d'úlceres pressió en AMB_PA i TOTAL.


In [380]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Úlceres pressió'], columns=TOTAL['Úlceres pressió'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i TOTAL.")


Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'úlceres pressió en AMB_PA_MECVV i TOTAL.


In [384]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Úlceres pressió'], columns=TOTAL['Úlceres pressió'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en SENSE_PA i TOTAL.")


Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'úlceres de pressió en SENSE_PA i TOTAL.


## Immobilitat (aqui)

In [78]:
imm_AMB_PA= (AMB_PA['Immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_AMB_PA:.2f}%")

KeyError: 'immobilitat'

In [79]:
imm_AMB_PA_MECVV= (AMB_PA_MECVV['immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_AMB_PA_MECVV:.2f}%")

KeyError: 'immobilitat'

In [80]:
imm_SENSE_PA= (SENSE_PA['immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_SENSE_PA:.2f}%")

KeyError: 'immobilitat'

#### Pvalor

##### Normalitat

In [81]:
stat, p_value = shapiro(AMB_PA['immobilitat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'immobilitat'

In [82]:
stat, p_value = shapiro(AMB_PA_MECVV['immobilitat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'immobilitat'

In [83]:
stat, p_value = shapiro(SENSE_PA['immobilitat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'immobilitat'

In [84]:
stat, p_value_12 = ttest_ind(AMB_PA['immobilitat'], AMB_PA_MECVV['immobilitat'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'immobilitat'

In [85]:
stat, p_value_13 = ttest_ind(AMB_PA['immobilitat'], SENSE_PA['immobilitat'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'immobilitat'

In [86]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['immobilitat'], SENSE_PA['immobilitat'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'immobilitat'

## Confusió

In [87]:
conf_AMB_PA= (AMB_PA['confusio'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_AMB_PA:.2f}%")

KeyError: 'confusio'

In [88]:
conf_AMB_PA_MECVV= (AMB_PA_MECVV['confusio'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_AMB_PA_MECVV:.2f}%")

KeyError: 'confusio'

In [89]:
conf_SENSE_PA= (SENSE_PA['confusio'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_SENSE_PA:.2f}%")

KeyError: 'confusio'

#### Pvalor

##### Normalitat

In [90]:
stat, p_value = shapiro(AMB_PA['confusio'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'confusio'

In [91]:
stat, p_value = shapiro(AMB_PA_MECVV['confusio'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'confusio'

In [92]:
stat, p_value = shapiro(SENSE_PA['confusio'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'confusio'

In [93]:
stat, p_value_12 = mannwhitneyu(AMB_PA['confusio'], AMB_PA_MECVV['confusio'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'confusio'

In [94]:
stat, p_value_13 = mannwhitneyu(AMB_PA['confusio'], SENSE_PA['confusio'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'confusio'

In [95]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['confusio'], SENSE_PA['confusio'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'confusio'

## Osteoporosi

In [96]:
ost_AMB_PA= (AMB_PA['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_AMB_PA:.2f}%")

KeyError: 'osteoporosis'

In [97]:
ost_AMB_PA_MECVV= (AMB_PA_MECVV['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_AMB_PA_MECVV:.2f}%")

KeyError: 'osteoporosis'

In [98]:
ost_SENSE_PA= (SENSE_PA['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_SENSE_PA:.2f}%")

KeyError: 'osteoporosis'

#### Pvalor

##### Normalitat

In [99]:
stat, p_value = shapiro(AMB_PA['osteoporosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'osteoporosis'

In [100]:
stat, p_value = shapiro(AMB_PA_MECVV['osteoporosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'osteoporosis'

In [101]:
stat, p_value = shapiro(SENSE_PA['osteoporosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'osteoporosis'

In [102]:
stat, p_value_12 = ttest_ind(AMB_PA['osteoporosis'], AMB_PA_MECVV['osteoporosis'], equal_var= False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'osteoporosis'

In [103]:
stat, p_value_13 = ttest_ind(AMB_PA['osteoporosis'], SENSE_PA['osteoporosis'], equal_var= False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'osteoporosis'

In [104]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['osteoporosis'], SENSE_PA['osteoporosis'], equal_var= False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'osteoporosis'

## Sarcopènia

In [105]:
sarc_AMB_PA= (AMB_PA['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_AMB_PA:.2f}%")

KeyError: 'sarcopenia'

In [106]:
sarc_AMB_PA_MECVV= (AMB_PA_MECVV['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_AMB_PA_MECVV:.2f}%")

KeyError: 'sarcopenia'

In [107]:
sarc_SENSE_PA= (SENSE_PA['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_SENSE_PA:.2f}%")

KeyError: 'sarcopenia'

#### Pvalor

##### Normalitat

In [108]:
stat, p_value = shapiro(AMB_PA['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'sarcopenia'

In [109]:
stat, p_value = shapiro(AMB_PA_MECVV['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'sarcopenia'

In [110]:
stat, p_value = shapiro(SENSE_PA['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'sarcopenia'

In [111]:
stat, p_value_12 = mannwhitneyu(AMB_PA['sarcopenia'], AMB_PA_MECVV['sarcopenia'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'sarcopenia'

In [112]:
stat, p_value_13 = mannwhitneyu(AMB_PA['sarcopenia'], SENSE_PA['sarcopenia'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'sarcopenia'

In [113]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['sarcopenia'], SENSE_PA['sarcopenia'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'sarcopenia'

## Trastorns del son

In [114]:
ts_AMB_PA= (AMB_PA['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_AMB_PA:.2f}%")

KeyError: 'problsueño'

In [115]:
ts_AMB_PA_MECVV= (AMB_PA_MECVV['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_AMB_PA_MECVV:.2f}%")

KeyError: 'problsueño'

In [116]:
ts_SENSE_PA= (SENSE_PA['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_SENSE_PA:.2f}%")

KeyError: 'problsueño'

#### Pvalor

##### Normalitat

In [117]:
stat, p_value = shapiro(AMB_PA['problsueño'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'problsueño'

In [118]:
stat, p_value = shapiro(AMB_PA_MECVV['problsueño'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'problsueño'

In [119]:
stat, p_value = shapiro(SENSE_PA['problsueño'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'problsueño'

In [120]:
stat, p_value_12 = ttest_ind(AMB_PA['problsueño'], AMB_PA_MECVV['problsueño'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'problsueño'

In [121]:
stat, p_value_13 = ttest_ind(AMB_PA['problsueño'], SENSE_PA['problsueño'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'problsueño'

In [122]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['problsueño'], SENSE_PA['problsueño'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'problsueño'

## Dolor crònic

In [123]:
dc_AMB_PA= (AMB_PA['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_AMB_PA:.2f}%")

KeyError: 'dolor_cron'

In [124]:
dc_AMB_PA_MECVV= (AMB_PA_MECVV['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_AMB_PA_MECVV:.2f}%")

KeyError: 'dolor_cron'

In [125]:
dc_SENSE_PA= (SENSE_PA['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_SENSE_PA:.2f}%")

KeyError: 'dolor_cron'

#### Pvalor

##### Normalitat

In [126]:
stat, p_value = shapiro(AMB_PA['dolor_cron'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'dolor_cron'

In [127]:
stat, p_value = shapiro(AMB_PA_MECVV['dolor_cron'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'dolor_cron'

In [128]:
stat, p_value = shapiro(SENSE_PA['dolor_cron'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'dolor_cron'

In [129]:
stat, p_value_12 = ttest_ind(AMB_PA['dolor_cron'], AMB_PA_MECVV['dolor_cron'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'dolor_cron'

In [130]:
stat, p_value_13 = ttest_ind(AMB_PA['dolor_cron'], SENSE_PA['dolor_cron'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'dolor_cron'

In [131]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['dolor_cron'], SENSE_PA['dolor_cron'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'dolor_cron'

## Iatrogènic

In [132]:
iatr_AMB_PA= (AMB_PA['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_AMB_PA:.2f}%")

KeyError: 'iatrogenico'

In [133]:
iatr_AMB_PA_MECVV= (AMB_PA_MECVV['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_AMB_PA_MECVV:.2f}%")

KeyError: 'iatrogenico'

In [134]:
iatr_SENSE_PA= (SENSE_PA['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_SENSE_PA:.2f}%")

KeyError: 'iatrogenico'

#### Pvalor

##### Normalitat

In [135]:
stat, p_value = shapiro(AMB_PA['iatrogenico'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'iatrogenico'

In [136]:
stat, p_value = shapiro(AMB_PA_MECVV['iatrogenico'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'iatrogenico'

In [137]:
stat, p_value = shapiro(SENSE_PA['iatrogenico'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'iatrogenico'

In [138]:
stat, p_value_12 = ttest_ind(AMB_PA['iatrogenico'], AMB_PA_MECVV['iatrogenico'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'iatrogenico'

In [139]:
stat, p_value_13 = ttest_ind(AMB_PA['iatrogenico'], SENSE_PA['iatrogenico'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'iatrogenico'

In [140]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['iatrogenico'], SENSE_PA['iatrogenico'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'iatrogenico'

## Restrenyiment

In [141]:
est_AMB_PA= (AMB_PA['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_AMB_PA:.2f}%")

KeyError: 'estreñimiento'

In [142]:
est_AMB_PA_MECVV= (AMB_PA_MECVV['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_AMB_PA_MECVV:.2f}%")

KeyError: 'estreñimiento'

In [143]:
est_SENSE_PA= (SENSE_PA['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_SENSE_PA:.2f}%")

KeyError: 'estreñimiento'

#### Pvalor

##### Normalitat

In [144]:
stat, p_value = shapiro(AMB_PA['estreñimiento'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'estreñimiento'

In [145]:
stat, p_value = shapiro(AMB_PA_MECVV['estreñimiento'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'estreñimiento'

In [146]:
stat, p_value = shapiro(SENSE_PA['estreñimiento'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'estreñimiento'

In [147]:
stat, p_value_12 = ttest_ind(AMB_PA['estreñimiento'], AMB_PA_MECVV['estreñimiento'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'estreñimiento'

In [148]:
stat, p_value_13 = ttest_ind(AMB_PA['estreñimiento'], SENSE_PA['estreñimiento'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'estreñimiento'

In [149]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['estreñimiento'], SENSE_PA['estreñimiento'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'estreñimiento'

# Historial mèdic/comorbilitats

## Malalties CV

In [150]:
cv_AMB_PA= (AMB_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_AMB_PA:.2f}%")

Percentatge de problemes CV: 5.88%


In [151]:
cv_AMB_PA_MECVV= (AMB_PA_MECVV['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_AMB_PA_MECVV:.2f}%")

Percentatge de problemes CV: 0.00%


In [152]:
cv_SENSE_PA= (SENSE_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_SENSE_PA:.2f}%")

Percentatge de problemes CV: 25.00%


#### Pvalor

##### Normalitat 

In [153]:
stat, p_value = shapiro(AMB_PA['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [154]:
stat, p_value = shapiro(AMB_PA_MECVV['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [155]:
stat, p_value = shapiro(SENSE_PA['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [156]:
stat, p_value_12 = mannwhitneyu(AMB_PA['CV'], AMB_PA_MECVV['CV'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.5483429354628137


In [157]:
stat, p_value_13 = mannwhitneyu(AMB_PA['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.19610818609418723


In [158]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.1699515583739133


## Malalties del cor

In [159]:
cora_AMB_PA= (AMB_PA['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_AMB_PA:.2f}%")

KeyError: 'probl_corazon'

In [160]:
cora_AMB_PA_MECVV= (AMB_PA_MECVV['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_AMB_PA_MECVV:.2f}%")

KeyError: 'probl_corazon'

In [161]:
cora_SENSE_PA= (SENSE_PA['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_SENSE_PA:.2f}%")

KeyError: 'probl_corazon'

#### Pvalor

##### Normalitat

In [162]:
stat, p_value = shapiro(AMB_PA['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'probl_corazon'

In [163]:
stat, p_value = shapiro(AMB_PA_MECVV['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'probl_corazon'

In [164]:
stat, p_value = shapiro(SENSE_PA['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'probl_corazon'

In [165]:
stat, p_value_12 = mannwhitneyu(AMB_PA['probl_corazon'], AMB_PA_MECVV['probl_corazon'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'probl_corazon'

In [166]:
stat, p_value_13 = mannwhitneyu(AMB_PA['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.19610818609418723


In [167]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.1699515583739133


## Malalties neurodegeneratives

In [168]:
nd_AMB_PA= (AMB_PA['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_AMB_PA:.2f}%")

KeyError: 'neurodegenerativas'

In [169]:
nd_AMB_PA_MECVV= (AMB_PA_MECVV['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_AMB_PA_MECVV:.2f}%")

KeyError: 'neurodegenerativas'

In [170]:
nd_SENSE_PA= (SENSE_PA['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_SENSE_PA:.2f}%")

KeyError: 'neurodegenerativas'

#### Pvalor

##### Normalitat

In [171]:
stat, p_value = shapiro(AMB_PA['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'neurodegenerativas'

In [172]:
stat, p_value = shapiro(AMB_PA_MECVV['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'neurodegenerativas'

In [173]:
stat, p_value = shapiro(SENSE_PA['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'neurodegenerativas'

In [174]:
stat, p_value_12 = mannwhitneyu(AMB_PA['neurodegenerativas'], AMB_PA_MECVV['neurodegenerativas'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'neurodegenerativas'

In [175]:
stat, p_value_13 = mannwhitneyu(AMB_PA['neurodegenerativas'], SENSE_PA['neurodegenerativas'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'neurodegenerativas'

In [176]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['neurodegenerativas'], SENSE_PA['neurodegenerativas'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'neurodegenerativas'

## Diabetes mellitus

In [177]:
dm_AMB_PA= (AMB_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_AMB_PA:.2f}%")

Percentatge de diabetes mellitus: 23.53%


In [178]:
dm_AMB_PA_MECVV= (AMB_PA_MECVV['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_AMB_PA_MECVV:.2f}%")

Percentatge de diabetes mellitus: 25.00%


In [179]:
dm_SENSE_PA= (SENSE_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_SENSE_PA:.2f}%")

Percentatge de diabetes mellitus: 25.00%


#### Pvalor 

##### Normalitat

In [180]:
stat, p_value = shapiro(AMB_PA['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [181]:
stat, p_value = shapiro(AMB_PA_MECVV['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [182]:
stat, p_value = shapiro(SENSE_PA['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [183]:
stat, p_value_12 = mannwhitneyu(AMB_PA['DM'], AMB_PA_MECVV['DM'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.9686157491081862


In [184]:
stat, p_value_13 = mannwhitneyu(AMB_PA['DM'], SENSE_PA['DM'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PAN", p_value_13)

P-valor entre AMB_PA i SENSE_PAN 0.9686157491081862


In [185]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['DM'], SENSE_PA['DM'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PAN", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PAN 1.0


## Hepatopaties

In [186]:
hp_AMB_PA= (AMB_PA['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_AMB_PA:.2f}%")

KeyError: 'hepatopatias'

In [187]:
hp_AMB_PA_MECVV= (AMB_PA_MECVV['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_AMB_PA_MECVV:.2f}%")

KeyError: 'hepatopatias'

In [188]:
hp_SENSE_PA= (SENSE_PA['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_SENSE_PA:.2f}%")

KeyError: 'hepatopatias'

#### Pvalor

##### Normalitat

In [189]:
stat, p_value = shapiro(AMB_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'hepatopatias'

In [190]:
stat, p_value = shapiro(AMB_PA_MECVV['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'hepatopatias'

In [191]:
stat, p_value = shapiro(SENSE_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'hepatopatias'

In [192]:
stat, p_value_12 = mannwhitneyu(AMB_PA['hepatopatias'], AMB_PA_MECVV['hepatopatias'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'hepatopatias'

In [193]:
stat, p_value_13 = mannwhitneyu(AMB_PA['hepatopatias'], SENSE_PA['hepatopatias'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'hepatopatias'

In [194]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['hepatopatias'], SENSE_PA['hepatopatias'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'hepatopatias'

## Neoplàsia

In [195]:
neopl_AMB_PA= (AMB_PA['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_AMB_PA:.2f}%")

KeyError: 'neoplasias'

In [196]:
neopl_AMB_PA_MECVV= (AMB_PA_MECVV['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_AMB_PA_MECVV:.2f}%")

KeyError: 'neoplasias'

In [197]:
neopl_SENSE_PA= (SENSE_PA['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_SENSE_PA:.2f}%")

KeyError: 'neoplasias'

#### Pvalor

##### Normalitat

In [198]:
stat, p_value = shapiro(AMB_PA['neoplasias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'neoplasias'

In [199]:
stat, p_value = shapiro(AMB_PA_MECVV['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'hepatopatias'

In [200]:
stat, p_value = shapiro(SENSE_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'hepatopatias'

In [201]:
stat, p_value_12 = mannwhitneyu(AMB_PA['neoplasias'], AMB_PA_MECVV['neoplasias'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'neoplasias'

In [202]:
stat, p_value_13 = mannwhitneyu(AMB_PA['neoplasias'], SENSE_PA['neoplasias'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_12)

KeyError: 'neoplasias'

In [203]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['neoplasias'], SENSE_PA['neoplasias'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'neoplasias'

## Malaltia crònica renal (creatinina > 1.5mg/dL)

Donat que aquesta columna és de tipus objecte, primer la passaré a tipus numèric per poder operar amb ella

In [204]:
AMB_PA.loc[:, 'creatinina'] = pd.to_numeric(AMB_PA['creatinina'], errors='coerce')
AMB_PA_MECVV.loc[:, 'creatinina'] = pd.to_numeric(AMB_PA_MECVV['creatinina'], errors='coerce')
SENSE_PA.loc[:, 'creatinina'] = pd.to_numeric(SENSE_PA['creatinina'], errors='coerce')

KeyError: 'creatinina'

In [205]:
ecr_AMB_PA= (AMB_PA['creatinina'] >= 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_AMB_PA:.2f}%")

KeyError: 'creatinina'

In [206]:
ecr_AMB_PA_MECVV= (AMB_PA_MECVV['creatinina'] > 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_AMB_PA_MECVV:.2f}%")

KeyError: 'creatinina'

In [207]:
ecr_SENSE_PA= (SENSE_PA['creatinina'] > 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_SENSE_PA:.2f}%")

KeyError: 'creatinina'

#### Pvalor

##### Normalitat

In [208]:
stat, p_value = shapiro(AMB_PA['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'creatinina'

In [209]:
stat, p_value = shapiro(AMB_PA_MECVV['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'creatinina'

In [210]:
stat, p_value = shapiro(SENSE_PA['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'creatinina'

In [211]:
stat, p_value_12 = ttest_ind(AMB_PA['creatinina'], AMB_PA_MECVV['creatinina'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'creatinina'

In [212]:
stat, p_value_13 = mannwhitneyu(AMB_PA['creatinina'], SENSE_PA['creatinina'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'creatinina'

In [213]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['creatinina'], SENSE_PA['creatinina'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'creatinina'

## Fallida renal aguda (o insuficiència renal aguda)

In [214]:
ira_AMB_PA= (AMB_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_AMB_PA:.2f}%")

Percentatge de fallida renal aguda: 23.53%


In [215]:
ira_AMB_PA_MECVV= (AMB_PA_MECVV['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_AMB_PA_MECVV:.2f}%")

Percentatge de fallida renal aguda: 50.00%


In [216]:
ira_SENSE_PA= (SENSE_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_SENSE_PA:.2f}%")

Percentatge de fallida renal aguda: 50.00%


#### Pvalor

##### Normalitat

In [217]:
stat, p_value = shapiro(AMB_PA['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [218]:
stat, p_value = shapiro(AMB_PA_MECVV['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6647
Valor p = 0.0009
El nombre de files no segueix una distribució normal.


In [219]:
stat, p_value = shapiro(SENSE_PA['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6647
Valor p = 0.0009
El nombre de files no segueix una distribució normal.


In [220]:
stat, p_value_12 = mannwhitneyu(AMB_PA['ARF'], AMB_PA_MECVV['ARF'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.2073928860736831


In [221]:
stat, p_value_13 = mannwhitneyu(AMB_PA['ARF'], SENSE_PA['ARF'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.2073928860736831


In [222]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['ARF'], SENSE_PA['ARF'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_23)

P-valor entre AMB_PA i SENSE_PA 1.0


## Marejos/síncope/lipotímia/reacció vasovagal

In [223]:
dizsyn_AMB_PA= (AMB_PA['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_AMB_PA:.2f}%")

KeyError: 'mareos'

In [224]:
dizsyn_AMB_PA_MECVV= (AMB_PA_MECVV['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_AMB_PA_MECVV:.2f}%")

KeyError: 'mareos'

In [225]:
dizsyn_SENSE_PA= (SENSE_PA['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_SENSE_PA:.2f}%")

KeyError: 'mareos'

#### Pvalor

##### Normalitat

In [226]:
stat, p_value = shapiro(AMB_PA['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'mareos'

In [227]:
stat, p_value = shapiro(AMB_PA_MECVV['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'mareos'

In [228]:
stat, p_value = shapiro(SENSE_PA['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'mareos'

In [229]:
stat, p_value_12 = ttest_ind(AMB_PA['mareos'], AMB_PA_MECVV['mareos'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'mareos'

In [230]:
stat, p_value_12 = mannwhitneyu(AMB_PA['mareos'], AMB_PA_MECVV['mareos'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'mareos'

In [231]:
stat, p_value_13 = mannwhitneyu(AMB_PA['mareos'], SENSE_PA['mareos'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'mareos'

In [232]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['mareos'], SENSE_PA['mareos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'mareos'

## VIH

In [233]:
vih_AMB_PA= (AMB_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_AMB_PA:.2f}%")

Percentatge de VIH: 0.00%


In [234]:
vih_AMB_PA_MECVV= (AMB_PA_MECVV['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_AMB_PA_MECVV:.2f}%")

Percentatge de VIH: 0.00%


In [235]:
vih_SENSE_PA= (SENSE_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_SENSE_PA:.2f}%")

Percentatge de VIH: 0.00%


#### Pvalor

##### Normalitat

In [236]:
stat, p_value = shapiro(AMB_PA['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [237]:
stat, p_value = shapiro(AMB_PA_MECVV['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [238]:
stat, p_value = shapiro(SENSE_PA['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [239]:
stat, p_value_12 = ttest_ind(AMB_PA['VIH'], AMB_PA_MECVV['VIH'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV nan


In [240]:
stat, p_value_13 = ttest_ind(AMB_PA['VIH'], SENSE_PA['VIH'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA nan


In [241]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['VIH'], SENSE_PA['VIH'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA nan


## Psicosi

In [242]:
psi_AMB_PA= (AMB_PA['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {vih_AMB_PA:.2f}%")

KeyError: 'psicosis'

In [243]:
psi_AMB_PA_MECVV= (AMB_PA_MECVV['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_AMB_PA_MECVV:.2f}%")

KeyError: 'psicosis'

In [244]:
psi_SENSE_PA= (SENSE_PA['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_SENSE_PA:.2f}%")

KeyError: 'psicosis'

#### Pvalor 

##### Normalitat

In [245]:
stat, p_value = shapiro(AMB_PA['psicosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'psicosis'

In [246]:
stat, p_value = shapiro(AMB_PA_MECVV['psicosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'psicosis'

In [247]:
stat, p_value = shapiro(SENSE_PA['psicosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'psicosis'

In [248]:
stat, p_value_12 = ttest_ind(AMB_PA['psicosis'], AMB_PA_MECVV['psicosis'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'psicosis'

In [249]:
stat, p_value_13 = ttest_ind(AMB_PA['psicosis'], SENSE_PA['psicosis'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'psicosis'

In [250]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['psicosis'], SENSE_PA['psicosis'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'psicosis'

## Deficiències nutricionals

In [251]:
nutri_AMB_PA= (AMB_PA['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_AMB_PA:.2f}%")

KeyError: 'def_nutri'

In [252]:
nutri_AMB_PA_MECVV= (AMB_PA_MECVV['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_AMB_PA_MECVV:.2f}%")

KeyError: 'def_nutri'

In [253]:
nutri_SENSE_PA= (SENSE_PA['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_SENSE_PA:.2f}%")

KeyError: 'def_nutri'

#### Pvalor

##### Normalitat

In [254]:
stat, p_value = shapiro(AMB_PA['def_nutri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'def_nutri'

In [255]:
stat, p_value = shapiro(AMB_PA_MECVV['def_nutri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'def_nutri'

In [256]:
stat, p_value = shapiro(SENSE_PA['def_nutri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'def_nutri'

In [257]:
stat, p_value_12 = ttest_ind(AMB_PA['def_nutri'], AMB_PA_MECVV['def_nutri'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'def_nutri'

In [258]:
stat, p_value_13 = ttest_ind(AMB_PA['def_nutri'], SENSE_PA['def_nutri'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'def_nutri'

In [259]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['def_nutri'], SENSE_PA['def_nutri'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'def_nutri'